# Programa que calcula previsão para o dia seguinte

In [1]:
print('ola mundo')
import tensorflow as tf
print(tf.__version__)

ola mundo


2023-01-18 00:47:43.814327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-18 00:47:43.814410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
urlRedes = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"
urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'

In [4]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas

In [5]:
def montaDados(rede):
    global dfDados
    global X
    urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia?qtdeDias={}".format(rede.qtdeDias1)
    print('conexao exemplos')
    print(urlExemplos)
    r = requests.get(urlExemplos)
    j = r.json()
    dfDados = pd.DataFrame.from_dict(j)
    print(dfDados.shape)
    dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
    X = np.stack(dfTratadoX.values)
    print("X")
    print(X.shape)
    X.head()
    dfDados.drop('campoX', inplace=True, axis=1)

In [6]:
def trataRede(rede):
    montaDados(rede)
    print('ID:' , rede['id'])
    nome_arquivo = '/root/executa-rede/redes/treino_rede' + str(rede['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(rede['id'])
    dfDados['tipoCompraVenda'] = rede['tipoCompraVenda']
    dfEntradas = incluiEntrada(rede)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    json = dfPrevisao.to_dict(orient='list')
    #response = requests.post(urlPrevisao, data=json)
    print(json)
    print(response)

In [7]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [8]:
print('conexao redes')
r = requests.get(urlRedes)
j = r.json()
dfRede = pd.DataFrame.from_dict(j)
print(dfRede.head())

conexao redes
    id  grupoAcaoId  periodoTreinoRedeId tipoExemploTreinoId  regraProjecaoId  \
0   43           12                    1                None               68   
1   77           12                    1                None               66   
2   79           12                    1                None               68   
3   80           12                    1                None               69   
4  276           12                    1                None               48   

   redeNeuralId  ativoPrevisao  limiteParaEntrada  qtdeLucroTeste  \
0             2              1               0.23               8   
1             1              1               0.26               9   
2             1              1               0.23               9   
3             1              1               0.30               8   
4             6              1               0.42               9   

   qtdePrejuizoTeste  ...         dataPrevisaoTeste  mediaValorTeste  \
0           

In [9]:
dfRede.apply(trataRede, axis='columns')

conexao exemplos
http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia?qtdeDias=120
(84, 3)
X
(84, 1800)


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [10]:
print(X)

[[1.07482  1.066906 1.063309 ... 0.996403 0.999281 1.      ]
 [2.189394 2.219697 2.189394 ... 1.       1.       1.      ]
 [1.498557 1.512266 1.50938  ... 0.997835 1.002165 1.      ]
 ...
 [0.691368 0.692408 0.691628 ... 0.99844  1.00052  1.      ]
 [1.063676 1.069465 1.069465 ... 0.994211 1.       1.      ]
 [1.444083 1.479913 1.471227 ... 1.0076   1.001086 1.      ]]
